<a href="https://colab.research.google.com/github/eduardofae/RL/blob/main/AT-05/05%20-%20TD%20learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Métodos de Diferença Temporal

Nesta tarefa, você irá implementar e comparar os algoritmos de Aprendizado por Reforço por Diferença Temporal SARSA e Q-Learning.

**Objetivos:**

1.  **Compreender a Diferença entre SARSA e Q-Learning:** Observe e implemente as regras de atualização de Q-values para ambos os algoritmos, prestando atenção especial à forma como cada um calcula o alvo na atualização de valor.
2.  **Implementar Agentes SARSA e Q-Learning:** Preencha as classes `SarsaAgent` e `QLearningAgent` para que implementem corretamente os algoritmos SARSA e Q-Learning, respectivamente. Certifique-se de que as funções `updateQ` e `train` estejam alinhadas com as especificações de cada algoritmo.
3.  **Testar no Grid 4x3:** Utilize os agentes implementados para resolver o ambiente Grid 4x3 fornecido. Observe as políticas aprendidas e os valores Q resultantes.
5.  **Testar no Cliff Walking:** Treine os agentes SARSA e Q-Learning no ambiente Cliff Walking.
6.  **Comparar Políticas no Cliff Walking:** Analise e compare as políticas ótimas aprendidas por SARSA e Q-Learning no ambiente Cliff Walking. **Observe atentamente as diferenças nas rotas preferidas pelos agentes.** Devido à natureza on-policy do SARSA e off-policy do Q-Learning, suas políticas ótimas no Cliff Walking (um ambiente com penalidade grande por cair do precipício) deverão ser notavelmente diferentes. SARSA tenderá a aprender uma política mais "segura", enquanto Q-Learning pode aprender uma política que, embora ótima em termos de valor Q, é mais arriscada durante o treinamento sob uma política epsilon-gulosa.
7.  **Visualizar Resultados:** Utilize o `AgentVisualizer` (adaptado para o ambiente Cliff Walking, se necessário) para visualizar as políticas e valores Q aprendidos em ambos os ambientes.

Ao final desta tarefa, você deverá ser capaz de explicar a diferença fundamental entre SARSA e Q-Learning e demonstrar como essa diferença se manifesta nas políticas aprendidas em um ambiente com riscos como o Cliff Walking.

## Grid 4x3

O código abaixo deve ser preenchido com a implementação do grid 4x3, com as mesmas especificações do Colab de [MDPs](https://colab.research.google.com/drive/1iNG9EX1-piXQvmN4-wQzncBv4EX7-yZV?usp=sharing). Portanto, você pode/deve aproveitar o código já feito. Não há mais aquelas funções adicionais para permitir que o ambiente seja "consultado" para execução de métodos de programação dinâmica, porque agora o agente não vai conhecer a dinâmica do ambiente. Ele irá interagir somente pela API padrão do gymnasium.


### Especificação do GridWorld (igual à tarefa de MDPs)

A célula abaixo tem a especificação do grid 4x3, igual no colab de MDPs. Pode pular se você já tiver feito.



Implemente o ambiente do grid 4x3, preenchendo as células abaixo. Você deve permitir ao usuário especificar a recompensa de cada passo (padrão = -0.04), a probabilidade de 'escorregar' (padrão = 0.2) e o numero máximo de passos antes de encerrar o episódio. O espaço de ações deve ser discreto, com 4 ações (0=UP, 1=RIGHT, 2=DOWN, 3=LEFT), e o de estados também será discreto, com 12 estados (mesmo o estado 'parede' pode ser considerado nesta contagem). A convenção para numeração dos estados é (G=goal, #=parede, P=pit/buraco,S=start):
```
y=2    +----+----+----+----+
       |  8 |  9 | 10 | 11G|
       +----+----+----+----+
y=1    |  4 |  5#|  6 |  7P|
       +----+----+----+----+
y=0    |  0S|  1 |  2 |  3 |
       +----+----+----+----+
        x=0   x=1   x=2   x=3
```

Note que há métodos para converter a numeração de estados para as coordenadas x,y

## Código do GridWorld 4x3

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from typing import Optional,Iterable,Tuple
import matplotlib.pyplot as plt


class GridWorld4x3(gym.Env):
    metadata = {"render_modes": ["ansi"]}

    def __init__(
        self,
        reward_step: float = -0.04,
        slip: float = 0.2,
        max_steps: int = 1000,
        seed: Optional[int] = None,
        render_mode: Optional[str] = None,
    ):
        super().__init__()
        self.ncols = 4
        self.nrows = 3
        self.observation_space = spaces.Discrete(self.ncols * self.nrows)
        self.action_space = spaces.Discrete(4)  # 0=up, 1=right, 2=down, 3=left

        self.reward_step = reward_step
        self.slip = slip
        self.max_steps = max_steps
        self.render_mode = render_mode

        self.start_pos = (0, 0)
        self.goal_pos = (3, 2)  # state 11
        self.pit_pos = (3, 1)   # state 7
        self.wall_pos = (1, 1)  # state 5 (inacessível)

        self._rng = np.random.default_rng(seed)
        self.steps = 0
        self.agent_pos = self.start_pos

        # Movimentos: up, right, down, left
        self.moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    # ---------- conversão estado/posição ----------
    def pos_to_state(self, pos):
        x, y = pos
        return y * self.ncols + x

    def state_to_pos(self, s):
        return (s % self.ncols, s // self.ncols)

    # ---------- helpers internos ----------
    def _move(self, pos, action):
        dx, dy = self.moves[action]
        x, y = pos
        new_pos = (x + dx, y + dy)
        # checa limites e parede
        if not (0 <= new_pos[0] < self.ncols and 0 <= new_pos[1] < self.nrows):
            return pos
        if new_pos == self.wall_pos:
            return pos
        return new_pos

    def _reward_and_done(self, pos):
        if pos == self.goal_pos:
            return 1.0, True
        elif pos == self.pit_pos:
            return -1.0, True
        return self.reward_step, False

    # ---------- API Gym ----------
    def reset(self, *, seed: Optional[int] = None, options: Optional[dict] = None):
        super().reset(seed=seed)
        self.agent_pos = self.start_pos
        self.steps = 0
        return self.pos_to_state(self.agent_pos), {}

    def step(self, action):
        self.steps += 1

        # sorteia se escorrega
        if self._rng.random() < self.slip:
            if action in [0, 2]:  # up/down → troca por left/right
                action = self._rng.choice([1, 3])
            else:  # left/right → troca por up/down
                action = self._rng.choice([0, 2])

        self.agent_pos = self._move(self.agent_pos, action)
        reward, terminated = self._reward_and_done(self.agent_pos)
        truncated = self.steps >= self.max_steps

        return self.pos_to_state(self.agent_pos), reward, terminated, truncated, {}

    # ----------------------------
    # Rendering
    # ----------------------------
    def render(self, mode="ansi"):
        if mode == "ansi":
            return self._render_ansi()
        else:
            raise NotImplementedError

    def _render_ansi(self):
        out = ""
        for y in reversed(range(self.nrows)):
            out += "+----" * self.ncols + "+\n"
            for x in range(self.ncols):
                pos = (x, y)
                s = self.pos_to_state(pos)
                cell = f"{s:2d} "
                if pos == self.wall_pos:
                    cell = " ## "
                elif pos == self.goal_pos:
                    cell = f"{s:2d}G"
                elif pos == self.pit_pos:
                    cell = f"{s:2d}P"
                elif pos == self.start_pos:
                    cell = f"{s:2d}S"
                if self.agent_pos == self.state_to_pos(s):
                    cell = f"[{cell.strip()}]"
                out += f"|{cell:4}"
            out += "|\n"
        out += "+----" * self.ncols + "+\n"
        return out



## Codigo pra testar o Env

A célula abaixo faz o teste básico pra verificar que o ambiente respeita a interface gymnasium

In [ ]:
# Deve continuar passando nos testes do Gymnasium
from gymnasium.utils.env_checker import check_env

# Criar uma instância do ambiente
env = GridWorld4x3()

# This will catch many common issues
try:
    check_env(env)
    print("Environment passes all checks!")
except Exception as e:
    print(f"Environment has issues: {e}")

Environment passes all checks!


## Agente de TD learning



Agora você irá implementar os algoritmos de Aprendizado por Reforço por Diferença Temporal (TD Learning). Você irá trabalhar em subclasses da classe base `TDAgent` fornecida abaixo. Seu objetivo é preencher as partes faltando para que os agentes SARSA e Q-Learning sejam capazes de escolher ações e atualizar estimativas de valor para encontrar a política ótima em um ambiente. No caso deste enunciado será o grid 4x3 e Cliff Walking.

Você irá:

1.  **Compreender a classe base `TDAgent`**: Analisar a classe base `TDAgent` para entender os métodos e atributos comuns aos agentes SARSA e Q-Learning.
2.  **Implementar Agentes SARSA e Q-Learning:** Preencher as classes `SarsaAgent` e `QLearningAgent` (que já estão na sequência do notebook) para que implementem corretamente os algoritmos SARSA e Q-Learning, respectivamente. Certifique-se de que as funções `updateQ` e `train` estejam alinhadas com as especificações de cada algoritmo.


### Código do TDAgent

In [ ]:
from abc import ABC,abstractmethod

class TDAgent(ABC):
    def __init__(self, env: gym.Env, alpha: float = 0.1, gamma: float = 0.99, epsilon: float = 0.1):
        """
        Construtor do agente TD.

        Args:
            env: ambiente Gymnasium (ex: gridworld 4x3).
            alpha: taxa de aprendizado.
            gamma: fator de desconto.
            epsilon: taxa de exploração (para política epsilon-greedy).
        """
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

        obs_space_size = env.observation_space.n
        act_space_size = env.action_space.n
        self.q_values = np.zeros((obs_space_size, act_space_size))

    def Q(self, state, action) -> float:
      """Retorna Q(s,a)."""
      return self.q_values[state, action]

    def V(self, state) -> float:
      """Retorna V(s) = max_a Q(s,a)."""
      return np.max(self.q_values[state, :])

    def greedy_action(self, state) -> int:
      """Retorna a ação gulosa (argmax_a Q(s,a))."""
      return np.argmax(self.q_values[state, :])

    def act(self, state) -> int:
      """Retorna ação epsilon-greedy."""
      if np.random.rand() < self.epsilon:
          return self.env.action_space.sample()
      else:
          return self.greedy_action(state)

    @abstractmethod
    def train(self, steps: int):
      """
      Executa o treinamento por um número de passos.

      Args:
          steps: número de passos de treino.
      """
      pass





## Código do SarsaAgent

In [ ]:
class SarsaAgent(TDAgent):
    def updateQ(self, s, a, r, s_next, a_next, done: bool):
        """Atualiza Q(s,a) segundo a regra do SARSA."""
        self.q_values[s, a] += self.alpha * (r + self.gamma * self.Q(s_next, a_next) - self.Q(s, a))

    def train(self, steps: int):
        state, _ = self.env.reset()
        action = self.act(state)
        for step in range(steps):
            new_state, reward, terminated, truncated, _ = self.env.step(action)
            new_action = self.act(new_state)
            done = terminated or truncated
            self.updateQ(state, action, reward, new_state, new_action, done)
            state = new_state
            action = new_action
            if done:
                state, _ = self.env.reset()
                action = self.act(state)


### Testes do SarsaAgent

In [ ]:
# ----------------------
# Testes para SARSA
# ----------------------
def test_sarsa_update():
    class DummyEnv:
        observation_space = type("obs", (), {"n": 2})
        action_space = type("act", (), {"n": 2})
    env = DummyEnv()

    agent = SarsaAgent(env, alpha=0.5, gamma=1.0, epsilon=0.0)
    agent.q_values[:] = 0.0

    s, a, r, s_next, a_next = 0, 0, 1, 1, 1
    agent.q_values[s_next, a_next] = 2.0  # valor da ação escolhida em s'

    # target = r + γ Q(s',a') = 1 + 2 = 3
    # update: 0 + 0.5*(3-0) = 1.5
    agent.updateQ(s, a, r, s_next, a_next, done=False)

    assert np.isclose(agent.q_values[s, a], 1.5)


def test_sarsa_update_terminal():
    class DummyEnv:
        observation_space = type("obs", (), {"n": 2})
        action_space = type("act", (), {"n": 2})
    env = DummyEnv()

    agent = SarsaAgent(env, alpha=1.0, gamma=1.0, epsilon=0.0)
    agent.q_values[:] = 0.0

    s, a, r, s_next, a_next = 0, 1, -1, 1, 0
    agent.updateQ(s, a, r, s_next, a_next, done=True)

    # Como done=True, target = r = -1
    assert np.isclose(agent.q_values[s, a], -1.0)


## Código do QLearningAgent

In [ ]:
class QLearningAgent(TDAgent):
    def updateQ(self, s, a, r, s_next, done: bool):
        """Atualiza Q(s,a) segundo a regra do Q-Learning."""
        self.q_values[s, a] += self.alpha * (r + self.gamma * self.Q(s_next, self.greedy_action(s_next)) - self.Q(s, a))

    def train(self, steps: int):
        state, _ = self.env.reset()
        for step in range(steps):
            action = self.act(state)
            new_state, reward, terminated, truncated, _ = self.env.step(action)
            done = terminated or truncated
            self.updateQ(state, action, reward, new_state, done)
            state = new_state
            if done:
                state, _ = self.env.reset()



### Teste do Q0learningAgent

In [ ]:
# ----------------------
# Testes para Q-Learning
# ----------------------
def test_qlearning_update():
    class DummyEnv:
        observation_space = type("obs", (), {"n": 2})
        action_space = type("act", (), {"n": 2})
    env = DummyEnv()

    agent = QLearningAgent(env, alpha=0.5, gamma=1.0, epsilon=0.0)
    agent.q_values[:] = 0.0

    s, a, r, s_next = 0, 0, 1, 1
    agent.q_values[s_next, 1] = 2.0  # valor ótimo em s'

    # target = r + γ max_a Q(s',a) = 1 + 2 = 3
    # update: Q(s,a) += α * (target - Q(s,a)) = 0 + 0.5*(3-0) = 1.5
    agent.updateQ(s, a, r, s_next, done=False)

    assert np.isclose(agent.q_values[s, a], 1.5)


def test_qlearning_update_terminal():
    class DummyEnv:
        observation_space = type("obs", (), {"n": 2})
        action_space = type("act", (), {"n": 2})
    env = DummyEnv()

    agent = QLearningAgent(env, alpha=1.0, gamma=1.0, epsilon=0.0)
    agent.q_values[:] = 0.0

    s, a, r, s_next = 0, 1, -1, 1
    agent.updateQ(s, a, r, s_next, done=True)

    # Como done=True, target = r = -1
    assert np.isclose(agent.q_values[s, a], -1.0)
test_qlearning_update()
test_qlearning_update_terminal()

## Utilitário para visualizar o agente no Grid 4x3

O mesmo da tarefa anterior.

### Código do utilitário

In [ ]:
class AgentVisualizer:
    def __init__(self, agent, env):
        """
        agent: ValueIterationAgent-like (tem V(s), Q(s,a) e greedy_action(s))
        env: GridWorld4x3-like (tem nrows, ncols, pos_to_state, state_to_pos, is_terminal, get_states, start_pos, goal_pos, pit_pos, wall_pos)
        """
        self.agent = agent
        self.env = env
        self.action_to_str = {0: "↑", 1: "→", 2: "↓", 3: "←"}

        # Precompute special states
        self.wall_s = self.env.pos_to_state(self.env.wall_pos)
        self.start_s = self.env.pos_to_state(self.env.start_pos)
        self.goal_s = self.env.pos_to_state(self.env.goal_pos)
        self.pit_s = self.env.pos_to_state(self.env.pit_pos)

    # -----------------------
    # Política (setas)
    # -----------------------
    def print_policy(self):
        rows, cols = self.env.nrows, self.env.ncols
        horiz = "+" + "+".join(["------"] * cols) + "+"

        for y in reversed(range(rows)):
            print(horiz)
            cells = []
            for x in range(cols):
                s = self.env.pos_to_state((x, y))
                if s == self.wall_s:
                    content = "##"
                elif s == self.goal_s:
                    content = " G "
                elif s == self.pit_s:
                    content = " P "
                else:
                    a = self.agent.greedy_action(s)
                    arrow = self.action_to_str.get(a, "?")
                    if s == self.start_s:
                        content = f"S{arrow}"
                    else:
                        content = arrow
                cells.append(f"{content:^6}")
            print("|" + "|".join(cells) + "|")
        print(horiz)

    # -----------------------
    # Valores V(s)
    # -----------------------
    def print_values(self):
        rows, cols = self.env.nrows, self.env.ncols
        horiz = "+" + "+".join(["--------"] * cols) + "+"

        for y in reversed(range(rows)):
            print(horiz)
            cells = []
            for x in range(cols):
                s = self.env.pos_to_state((x, y))
                if s == self.wall_s:
                    content = "####"
                else:
                    v = self.agent.V(s)
                    if s == self.goal_s:
                        content = f"G({v:.2f})"
                    elif s == self.pit_s:
                        content = f"P({v:.2f})"
                    else:
                        content = f"{v:6.2f}"
                cells.append(f"{content:^8}")
            print("|" + "|".join(cells) + "|")
        print(horiz)

    # -----------------------
    # Q-values
    # -----------------------
    def print_qvalues(self):
        rows, cols = self.env.nrows, self.env.ncols
        horiz = "+" + "+".join(["---------------"] * cols) + "+"

        for y in reversed(range(rows)):
            print(horiz)
            # três linhas por célula
            line1, line2, line3 = [], [], []
            for x in range(cols):
                s = self.env.pos_to_state((x, y))
                if s == self.wall_s:
                    c1 = "###############"
                    c2 = "###############"
                    c3 = "###############"
                else:
                    qvals = [self.agent.Q(s, a) for a in range(4)]
                    best = int(np.argmax(qvals))
                    up = f"↑:{qvals[0]:.2f}"
                    left = f"←:{qvals[3]:.2f}"
                    right = f"→:{qvals[1]:.2f}"
                    down = f"↓:{qvals[2]:.2f}"
                    c1 = f"{up:^15}"
                    c2 = f"{left:<7}{right:>8}"
                    c3 = f"{down:^15}"
                line1.append(c1)
                line2.append(c2)
                line3.append(c3)

            # agora cada linha recebe delimitadores
            print("|" + "|".join(line1) + "|")
            print("|" + "|".join(line2) + "|")
            print("|" + "|".join(line3) + "|")
        print(horiz)


### Q-learning no 4x3

```
=== Política Aprendida (Q-Learning) ===
+------+------+------+------+
|  →   |  →   |  →   |  G   |
+------+------+------+------+
|  ↑   |  ##  |  ↑   |  P   |
+------+------+------+------+
|  S↑  |  ←   |  ↑   |  ←   |
+------+------+------+------+

=== Valores de Estado V(s) (Q-Learning) ===
+--------+--------+--------+--------+
|   0.81 |   0.88 |   0.96 |G(0.00) |
+--------+--------+--------+--------+
|   0.75 |  ####  |   0.64 |P(0.00) |
+--------+--------+--------+--------+
|   0.70 |   0.64 |   0.43 |   0.03 |
+--------+--------+--------+--------+

=== Q-values (Q-Learning) ===
+---------------+---------------+---------------+---------------+
|    ↑:0.73     |    ↑:0.82     |    ↑:0.87     |    ↑:0.00     |
|←:0.75   →:0.81|←:0.78   →:0.88|←:0.79   →:0.96|←:0.00   →:0.00|
|    ↓:0.73     |    ↓:0.81     |    ↓:0.66     |    ↓:0.00     |
+---------------+---------------+---------------+---------------+
|    ↑:0.75     |###############|    ↑:0.64     |    ↑:0.00     |
|←:0.70   →:0.69|###############|←:0.35  →:-0.61|←:0.00   →:0.00|
|    ↓:0.64     |###############|    ↓:0.10     |    ↓:0.00     |
+---------------+---------------+---------------+---------------+
|    ↑:0.70     |    ↑:0.38     |    ↑:0.43     |    ↑:-0.34    |
|←:0.64   →:0.57|←:0.64   →:0.08|←:-0.03 →:-0.08|←:0.03  →:-0.21|
|    ↓:0.61     |    ↓:0.29     |    ↓:-0.07    |    ↓:-0.17    |
+---------------+---------------+---------------+---------------+
```

In [ ]:
# =======================
# Execução do Q-Learning no Grid 4x3
# =======================

env_grid = GridWorld4x3()
agent_q_grid = QLearningAgent(env_grid, alpha=0.1, gamma=0.99, epsilon=0.1)

print("Treinando Q-Learning no Grid 4x3 por 10.000 passos...")
agent_q_grid.train(steps=10000)
print("Treinamento concluído.")

viz_grid_q = AgentVisualizer(agent_q_grid, env_grid)

print("\n=== Política Aprendida (Q-Learning) ===")
viz_grid_q.print_policy()

print("\n=== Valores de Estado V(s) (Q-Learning) ===")
viz_grid_q.print_values()

print("\n=== Q-values (Q-Learning) ===")
viz_grid_q.print_qvalues()

Treinando Q-Learning no Grid 4x3 por 10.000 passos...
Treinamento concluído.

=== Política Aprendida (Q-Learning) ===
+------+------+------+------+
|  →   |  →   |  →   |  G   |
+------+------+------+------+
|  ↑   |  ##  |  ↑   |  P   |
+------+------+------+------+
|  S↑  |  ←   |  ←   |  ←   |
+------+------+------+------+

=== Valores de Estado V(s) (Q-Learning) ===
+--------+--------+--------+--------+
|   0.81 |   0.88 |   0.96 |G(0.00) |
+--------+--------+--------+--------+
|   0.74 |  ####  |   0.54 |P(0.00) |
+--------+--------+--------+--------+
|   0.66 |   0.59 |   0.42 |   0.43 |
+--------+--------+--------+--------+

=== Q-values (Q-Learning) ===
+---------------+---------------+---------------+---------------+
|    ↑:0.64     |    ↑:0.77     |    ↑:0.81     |    ↑:0.00     |
|←:0.68   →:0.81|←:0.65   →:0.88|←:0.76   →:0.96|←:0.00   →:0.00|
|    ↓:0.66     |    ↓:0.74     |    ↓:0.59     |    ↓:0.00     |
+---------------+---------------+---------------+---------------+


### Sarsa no 4x3

In [ ]:
# =======================
# Execução do SARSA no Grid 4x3
# =======================

env_grid = GridWorld4x3()
agent_sarsa_grid = SarsaAgent(env_grid, alpha=0.1, gamma=0.99, epsilon=0.1)

print("Treinando SARSA no Grid 4x3 por 10.000 passos...")
agent_sarsa_grid.train(steps=10000)
print("Treinamento concluído.")

viz_grid_sarsa = AgentVisualizer(agent_sarsa_grid, env_grid)

print("\n=== Política Aprendida (SARSA) ===")
viz_grid_sarsa.print_policy()

print("\n=== Valores de Estado V(s) (SARSA) ===")
viz_grid_sarsa.print_values()

print("\n=== Q-values (SARSA) ===")
viz_grid_sarsa.print_qvalues()

Treinando SARSA no Grid 4x3 por 10.000 passos...
Treinamento concluído.

=== Política Aprendida (SARSA) ===
+------+------+------+------+
|  →   |  →   |  →   |  G   |
+------+------+------+------+
|  ↑   |  ##  |  ↑   |  P   |
+------+------+------+------+
|  S↑  |  ←   |  ←   |  ←   |
+------+------+------+------+

=== Valores de Estado V(s) (SARSA) ===
+--------+--------+--------+--------+
|   0.82 |   0.89 |   0.97 |G(0.00) |
+--------+--------+--------+--------+
|   0.74 |  ####  |   0.78 |P(0.00) |
+--------+--------+--------+--------+
|   0.65 |   0.53 |   0.36 |   0.19 |
+--------+--------+--------+--------+

=== Q-values (SARSA) ===
+---------------+---------------+---------------+---------------+
|    ↑:0.69     |    ↑:0.79     |    ↑:0.86     |    ↑:0.00     |
|←:0.65   →:0.82|←:0.67   →:0.89|←:0.79   →:0.97|←:0.00   →:0.00|
|    ↓:0.64     |    ↓:0.69     |    ↓:0.55     |    ↓:0.00     |
+---------------+---------------+---------------+---------------+
|    ↑:0.74     |###

## Ambiente CliffWalking

Este ambiente é um gridworld clássico utilizado para ilustrar as diferenças entre algoritmos on-policy e off-policy, como SARSA e Q-Learning.

**Descrição:**

*   É um grid 4x12.
*   O agente começa no canto inferior esquerdo (S - Start).
*   O objetivo é alcançar o canto inferior direito (G - Goal).
*   A área entre o início e o objetivo na linha inferior é um "penhasco" (Cliff).
*   Cair do penhasco resulta em uma grande penalidade (-100) e o agente retorna ao estado inicial (S).
*   Qualquer outro movimento resulta em uma pequena penalidade (-1).
*   O episódio termina ao alcançar o objetivo ou cair do penhasco.

Este ambiente destaca a exploração vs. segurança: um agente on-policy (como SARSA) que aprende e segue a mesma política exploratória tende a encontrar um caminho mais seguro longe do penhasco para minimizar o risco durante o treinamento. Um agente off-policy (como Q-Learning) que aprende a política ótima independentemente da política de exploração pode encontrar um caminho "ótimo" mais arriscado (próximo ao penhasco) se as transições diretas para o objetivo tiverem valores Q altos, mesmo que a política exploratória frequentemente o leve ao penhasco.

### Código do CliffWalkingEnv

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
from typing import Optional

class CliffWalkingEnv(gym.Env):
    metadata = {"render_modes": ["ansi"]}

    def __init__(self, render_mode: Optional[str] = None):
        super().__init__()
        self.nrows = 4
        self.ncols = 12
        self.observation_space = spaces.Discrete(self.nrows * self.ncols)
        self.action_space = spaces.Discrete(4)  # 0: up, 1: right, 2: down, 3: left

        self.start_pos = (0, 0)
        self.goal_pos = (11, 0)
        self.cliff_rows = [0]
        self.cliff_cols = list(range(1, 11))

        self.render_mode = render_mode
        self.agent_pos = self.start_pos

        # Define movements: up, right, down, left
        self.moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    def pos_to_state(self, pos):
        x, y = pos
        return y * self.ncols + x

    def state_to_pos(self, s):
        return (s % self.ncols, s // self.ncols)

    def _is_cliff(self, pos):
        x, y = pos
        return y in self.cliff_rows and x in self.cliff_cols

    def _is_goal(self, pos):
        return pos == self.goal_pos

    def reset(self, *, seed: Optional[int] = None, options: Optional[dict] = None):
        super().reset(seed=seed)
        self.agent_pos = self.start_pos
        return self.pos_to_state(self.agent_pos), {}

    def step(self, action):
        x, y = self.agent_pos
        dx, dy = self.moves[action]
        new_pos = (x + dx, y + dy)

        # Check for boundary conditions
        if not (0 <= new_pos[0] < self.ncols and 0 <= new_pos[1] < self.nrows):
            new_pos = self.agent_pos # Stay in place if hit a wall

        self.agent_pos = new_pos

        reward = -1 # default reward
        terminated = False
        truncated = False
        info = {}

        if self._is_cliff(self.agent_pos):
            reward = -100
            terminated = True
            self.agent_pos = self.start_pos # Return to start after falling
        elif self._is_goal(self.agent_pos):
            reward = 0
            terminated = True


        return self.pos_to_state(self.agent_pos), reward, terminated, truncated, info

    def render(self, mode="ansi"):
        if mode == "ansi":
            return self._render_ansi()
        else:
            super().render(mode=mode)

    def _render_ansi(self):
        output = ""
        for r in range(self.nrows - 1, -1, -1): # Iterate from top row down
            output += "+" + "---+" * self.ncols + "\n"
            row_str = "|"
            for c in range(self.ncols):
                pos = (c, r)
                if pos == self.agent_pos:
                    row_str += " A |"
                elif self._is_cliff(pos):
                    row_str += " C |"
                elif self._is_goal(pos):
                    row_str += " G |"
                elif pos == self.start_pos:
                     row_str += " S |"
                else:
                    row_str += "   |"
            output += row_str + "\n"
        output += "+" + "---+" * self.ncols + "\n"
        return output


## Visualizador pro CliffWalkingEnv

É bem parecido com o do Grid 4x3, apenas com as dimensões adaptadas

In [ ]:
import numpy as np

class CliffAgentVisualizer:
    def __init__(self, agent, env):
        """
        agent: agente com V(s), Q(s,a) e greedy_action(s)
        env: CliffWalkingEnv-like (tem nrows, ncols, pos_to_state, state_to_pos, start_pos, goal_pos, _is_cliff)
        """
        self.agent = agent
        self.env = env
        self.action_to_str = {0: "↑", 1: "→", 2: "↓", 3: "←"}

        # Precompute special states
        self.start_s = self.env.pos_to_state(self.env.start_pos)
        self.goal_s = self.env.pos_to_state(self.env.goal_pos)
        self.cliff_states = [self.env.pos_to_state((x, y))
                             for y in self.env.cliff_rows
                             for x in self.env.cliff_cols]

    # -----------------------
    # Política (setas)
    # -----------------------
    def print_policy(self):
        rows, cols = self.env.nrows, self.env.ncols
        horiz = "+" + "+".join(["------"] * cols) + "+"

        for y in reversed(range(rows)):
            print(horiz)
            cells = []
            for x in range(cols):
                s = self.env.pos_to_state((x, y))
                if s in self.cliff_states:
                    content = " C "
                elif s == self.goal_s:
                    content = " G "
                else:
                    a = self.agent.greedy_action(s)
                    arrow = self.action_to_str.get(a, "?")
                    if s == self.start_s:
                        content = f"S{arrow}"
                    else:
                        content = arrow
                cells.append(f"{content:^6}")
            print("|" + "|".join(cells) + "|")
        print(horiz)

    # -----------------------
    # Valores V(s)
    # -----------------------
    def print_values(self):
        rows, cols = self.env.nrows, self.env.ncols
        horiz = "+" + "+".join(["--------"] * cols) + "+"

        for y in reversed(range(rows)):
            print(horiz)
            cells = []
            for x in range(cols):
                s = self.env.pos_to_state((x, y))
                if s in self.cliff_states:
                    content = f"C(---)"
                else:
                    v = self.agent.V(s)
                    if s == self.goal_s:
                        content = f"G({v:.2f})"
                    elif s == self.start_s:
                        content = f"S({v:.1f})"
                    else:
                        content = f"{v:6.2f}"
                cells.append(f"{content:^8}")
            print("|" + "|".join(cells) + "|")
        print(horiz)

    # -----------------------
    # Q-values
    # -----------------------
    def print_qvalues(self):
        rows, cols = self.env.nrows, self.env.ncols
        horiz = "+" + "+".join(["---------------"] * cols) + "+"

        for y in reversed(range(rows)):
            print(horiz)
            # três linhas por célula
            line1, line2, line3 = [], [], []
            for x in range(cols):
                s = self.env.pos_to_state((x, y))
                if s in self.cliff_states:
                    c1 = " " * 15
                    c2 = "#### CLIFF ####"
                    c3 = " " * 15
                else:
                    qvals = [self.agent.Q(s, a) for a in range(4)]
                    up = f"↑:{qvals[0]:.2f}"
                    right = f"→:{qvals[1]:.1f}"
                    down = f"↓:{qvals[2]:.2f}"
                    left = f"←:{qvals[3]:.1f}"
                    c1 = f"{up:^15}"
                    c2 = f"{left:<7}{right:>8}"
                    c3 = f"{down:^15}"
                line1.append(c1)
                line2.append(c2)
                line3.append(c3)

            print("|" + "|".join(line1) + "|")
            print("|" + "|".join(line2) + "|")
            print("|" + "|".join(line3) + "|")
        print(horiz)


## Execução do Q-learning no CliffWalking

Executar o código a seguir deve gerar saídas parecidas como a abaixo (os valores não precisam ser idênticos, mas a política gerada deve passar na beira do penhasco, a partir do estado inicial).

```
=== Política Aprendida ===
+------+------+------+------+------+------+------+------+------+------+------+------+
|  →   |  ↓   |  ↓   |  ↓   |  →   |  ↓   |  ↓   |  →   |  →   |  →   |  ↓   |  ↓   |
+------+------+------+------+------+------+------+------+------+------+------+------+
|  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  ↓   |
+------+------+------+------+------+------+------+------+------+------+------+------+
|  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  ↓   |
+------+------+------+------+------+------+------+------+------+------+------+------+
|  S↑  |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  G   |
+------+------+------+------+------+------+------+------+------+------+------+------+

=== Valores de Estado V(s) ===
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| -12.50 | -11.82 | -10.99 | -10.00 |  -9.00 |  -8.00 |  -7.00 |  -6.00 |  -5.00 |  -4.00 |  -3.00 |  -2.00 |
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| -12.00 | -11.00 | -10.00 |  -9.00 |  -8.00 |  -7.00 |  -6.00 |  -5.00 |  -4.00 |  -3.00 |  -2.00 |  -1.00 |
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| -11.00 | -10.00 |  -9.00 |  -8.00 |  -7.00 |  -6.00 |  -5.00 |  -4.00 |  -3.00 |  -2.00 |  -1.00 |   0.00 |
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|S(-12.0)| C(---) | C(---) | C(---) | C(---) | C(---) | C(---) | C(---) | C(---) | C(---) | C(---) |G(0.00) |
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+

=== Q-values ===
+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|   ↑:-12.57    |   ↑:-12.27    |   ↑:-11.68    |   ↑:-10.00    |    ↑:-9.34    |    ↑:-8.60    |    ↑:-7.00    |    ↑:-6.75    |    ↑:-5.91    |    ↑:-4.95    |    ↑:-3.81    |    ↑:-2.87    |
|←:-12.6 →:-12.5|←:-11.8 →:-11.8|←:-11.4 →:-11.0|←:-11.0 →:-10.0|←:-10.5  →:-9.0|←:-8.5   →:-8.0|←:-8.7   →:-7.0|←:-7.8   →:-6.0|←:-6.9   →:-5.0|←:-5.8   →:-4.0|←:-4.9   →:-3.0|←:-3.8   →:-2.9|
|   ↓:-12.58    |   ↓:-11.82    |   ↓:-10.99    |   ↓:-10.00    |    ↓:-9.00    |    ↓:-8.00    |    ↓:-7.00    |    ↓:-6.00    |    ↓:-5.00    |    ↓:-4.00    |    ↓:-3.00    |    ↓:-2.00    |
+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|   ↑:-13.19    |   ↑:-12.56    |   ↑:-11.95    |   ↑:-11.00    |   ↑:-10.00    |    ↑:-9.00    |    ↑:-8.00    |    ↑:-7.00    |    ↑:-6.00    |    ↑:-5.00    |    ↑:-4.00    |    ↑:-3.00    |
|←:-13.0 →:-12.0|←:-13.0 →:-11.0|←:-12.0 →:-10.0|←:-11.0  →:-9.0|←:-10.0  →:-8.0|←:-9.0   →:-7.0|←:-8.0   →:-6.0|←:-7.0   →:-5.0|←:-6.0   →:-4.0|←:-5.0   →:-3.0|←:-4.0   →:-2.0|←:-3.0   →:-2.0|
|   ↓:-12.00    |   ↓:-11.00    |   ↓:-10.00    |    ↓:-9.00    |    ↓:-8.00    |    ↓:-7.00    |    ↓:-6.00    |    ↓:-5.00    |    ↓:-4.00    |    ↓:-3.00    |    ↓:-2.00    |    ↓:-1.00    |
+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|   ↑:-13.00    |   ↑:-12.00    |   ↑:-11.00    |   ↑:-10.00    |    ↑:-9.00    |    ↑:-8.00    |    ↑:-7.00    |    ↑:-6.00    |    ↑:-5.00    |    ↑:-4.00    |    ↑:-3.00    |    ↑:-2.00    |
|←:-12.0 →:-11.0|←:-12.0 →:-10.0|←:-11.0  →:-9.0|←:-10.0  →:-8.0|←:-9.0   →:-7.0|←:-8.0   →:-6.0|←:-7.0   →:-5.0|←:-6.0   →:-4.0|←:-5.0   →:-3.0|←:-4.0   →:-2.0|←:-3.0   →:-1.0|←:-2.0   →:-1.0|
|   ↓:-13.00    |   ↓:-100.00   |   ↓:-100.00   |   ↓:-100.00   |   ↓:-100.00   |   ↓:-100.00   |   ↓:-100.00   |   ↓:-100.00   |   ↓:-100.00   |   ↓:-100.00   |   ↓:-100.00   |    ↓:0.00     |
+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|   ↑:-12.00    |               |               |               |               |               |               |               |               |               |               |    ↑:0.00     |
|←:-13.0→:-100.0|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|←:0.0     →:0.0|
|   ↓:-13.00    |               |               |               |               |               |               |               |               |               |               |    ↓:0.00     |
+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
```

In [ ]:
# =======================
# Exemplo de uso
# =======================

env = CliffWalkingEnv()
agent = QLearningAgent(env, alpha=0.5, gamma=1.0, epsilon=0.1)

agent.train(steps=100_000)

viz = CliffAgentVisualizer(agent, env)

print("\n=== Política Aprendida ===")
viz.print_policy()

print("\n=== Valores de Estado V(s) ===")
viz.print_values()

print("\n=== Q-values ===")
viz.print_qvalues()



=== Política Aprendida ===
+------+------+------+------+------+------+------+------+------+------+------+------+
|  ←   |  →   |  ↓   |  →   |  ↓   |  →   |  ↓   |  →   |  →   |  →   |  ↓   |  ↓   |
+------+------+------+------+------+------+------+------+------+------+------+------+
|  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  ↓   |
+------+------+------+------+------+------+------+------+------+------+------+------+
|  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  ↓   |
+------+------+------+------+------+------+------+------+------+------+------+------+
|  S↑  |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  G   |
+------+------+------+------+------+------+------+------+------+------+------+------+

=== Valores de Estado V(s) ===
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| -12.27 | -11.78 | -10.98 | -10.00 |  -9.00 |  -8.00 | 

## Execução do Sarsa no CliffWalking

Executar o código a seguir deve gerar saídas parecidas como a abaixo. Os valores não precisam ser idênticos, mas a política gerada deve passar longe do penhasco, a partir do estado inicial. A política para os outros estados pode variar, já que eles serão bem menos visitados.

```
=== Política Aprendida ===
+------+------+------+------+------+------+------+------+------+------+------+------+
|  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  →   |  ↓   |
+------+------+------+------+------+------+------+------+------+------+------+------+
|  ↑   |  ↑   |  ↑   |  ↑   |  ↑   |  ↑   |  ↑   |  →   |  ↑   |  ↑   |  →   |  ↓   |
+------+------+------+------+------+------+------+------+------+------+------+------+
|  ↑   |  ↑   |  ↑   |  ↑   |  ↑   |  ↑   |  ↑   |  →   |  ↑   |  ↑   |  →   |  ↓   |
+------+------+------+------+------+------+------+------+------+------+------+------+
|  S↑  |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  G   |
+------+------+------+------+------+------+------+------+------+------+------+------+

=== Valores de Estado V(s) ===
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| -16.05 | -13.79 | -12.34 | -11.76 | -11.41 | -10.80 |  -9.45 |  -7.57 |  -7.37 |  -4.27 |  -3.12 |  -2.02 |
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| -18.89 | -19.29 | -15.12 | -18.39 | -15.25 | -12.36 | -12.33 |  -8.42 | -11.06 |  -6.35 |  -2.32 |  -1.00 |
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| -18.55 | -21.70 | -18.64 | -18.93 | -16.93 | -18.94 | -17.63 | -16.77 | -12.81 | -11.04 |  -1.00 |   0.00 |
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
|S(-19.5)| C(---) | C(---) | C(---) | C(---) | C(---) | C(---) | C(---) | C(---) | C(---) | C(---) |G(0.00) |
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+

=== Q-values ===
+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|   ↑:-19.52    |   ↑:-19.58    |   ↑:-17.73    |   ↑:-17.72    |   ↑:-16.35    |   ↑:-16.39    |   ↑:-15.75    |   ↑:-13.62    |   ↑:-10.74    |    ↑:-7.44    |    ↑:-6.55    |    ↑:-6.18    |
|←:-18.4 →:-16.0|←:-19.0 →:-13.8|←:-17.8 →:-12.3|←:-18.8 →:-11.8|←:-16.7 →:-11.4|←:-17.0 →:-10.8|←:-14.6  →:-9.5|←:-13.6  →:-7.6|←:-13.0  →:-7.4|←:-11.5  →:-4.3|←:-7.7   →:-3.1|←:-6.0   →:-4.9|
|   ↓:-20.60    |   ↓:-18.65    |   ↓:-18.46    |   ↓:-18.39    |   ↓:-17.65    |   ↓:-16.39    |   ↓:-14.56    |   ↓:-11.11    |   ↓:-10.87    |    ↓:-7.97    |    ↓:-8.14    |    ↓:-2.02    |
+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|   ↑:-18.89    |   ↑:-19.29    |   ↑:-15.12    |   ↑:-18.39    |   ↑:-15.25    |   ↑:-12.36    |   ↑:-12.33    |   ↑:-14.80    |   ↑:-11.06    |    ↑:-6.35    |    ↑:-6.24    |    ↑:-4.38    |
|←:-19.5 →:-19.6|←:-24.0 →:-19.7|←:-21.2 →:-21.9|←:-26.8 →:-36.6|←:-21.1 →:-23.5|←:-17.0 →:-18.5|←:-17.6 →:-13.4|←:-14.9  →:-8.4|←:-17.2 →:-15.5|←:-13.5 →:-15.4|←:-9.5   →:-2.3|←:-3.8   →:-3.8|
|   ↓:-21.98    |   ↓:-25.38    |   ↓:-22.27    |   ↓:-33.76    |   ↓:-67.16    |   ↓:-20.31    |   ↓:-17.57    |   ↓:-16.80    |   ↓:-37.44    |   ↓:-13.18    |   ↓:-14.37    |    ↓:-1.00    |
+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|   ↑:-18.55    |   ↑:-21.70    |   ↑:-18.64    |   ↑:-18.93    |   ↑:-16.93    |   ↑:-18.94    |   ↑:-17.63    |   ↑:-16.93    |   ↑:-12.81    |   ↑:-11.04    |   ↑:-13.64    |    ↑:-4.26    |
|←:-21.5 →:-27.2|←:-21.9 →:-24.8|←:-31.8 →:-35.6|←:-55.7 →:-31.2|←:-21.6 →:-21.3|←:-51.3 →:-31.3|←:-20.3 →:-27.9|←:-24.3 →:-16.8|←:-16.6 →:-33.7|←:-50.4 →:-51.7|←:-9.3   →:-1.0|←:-2.1   →:-1.0|
|   ↓:-23.40    |   ↓:-99.99    |   ↓:-98.44    |   ↓:-96.88    |   ↓:-98.44    |   ↓:-50.00    |   ↓:-87.50    |   ↓:-87.50    |   ↓:-98.44    |   ↓:-99.61    |   ↓:-99.99    |    ↓:0.00     |
+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+
|   ↑:-19.50    |               |               |               |               |               |               |               |               |               |               |    ↑:0.00     |
|←:-31.9→:-100.0|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|#### CLIFF ####|←:0.0     →:0.0|
|   ↓:-27.32    |               |               |               |               |               |               |               |               |               |               |    ↓:0.00     |
+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+


```

In [ ]:
# =======================
# Exemplo de uso
# =======================

env = CliffWalkingEnv()
agent = SarsaAgent(env, alpha=0.5, gamma=1.0, epsilon=0.1)

agent.train(steps=100_000)

viz = CliffAgentVisualizer(agent, env)

print("\n=== Política Aprendida ===")
viz.print_policy()

print("\n=== Valores de Estado V(s) ===")
viz.print_values()

print("\n=== Q-values ===")
viz.print_qvalues()



=== Política Aprendida ===
+------+------+------+------+------+------+------+------+------+------+------+------+
|  →   |  →   |  ↓   |  ↓   |  →   |  →   |  →   |  →   |  ↓   |  ↓   |  →   |  ↓   |
+------+------+------+------+------+------+------+------+------+------+------+------+
|  ↑   |  ↑   |  →   |  →   |  ↑   |  ↑   |  →   |  ↑   |  →   |  →   |  →   |  ↓   |
+------+------+------+------+------+------+------+------+------+------+------+------+
|  ←   |  ↑   |  ↑   |  →   |  ↑   |  ↑   |  ↑   |  ↑   |  ↑   |  ↑   |  →   |  ↓   |
+------+------+------+------+------+------+------+------+------+------+------+------+
|  S↑  |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  C   |  G   |
+------+------+------+------+------+------+------+------+------+------+------+------+

=== Valores de Estado V(s) ===
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| -25.08 | -23.86 | -22.44 | -22.51 | -16.29 | -13.35 | 